In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import cv2 as cv

#from google.colab.patches import cv2_imshow
from collections import OrderedDict
import cv2 as cv
from PIL import Image
import copy
import matplotlib.pyplot as plt
#importing all libraries

In [ ]:
img_size = 512
prep = transforms.Compose([transforms.Resize((img_size,img_size)),
                           transforms.ToTensor(),
                          # transforms.Lambda(lambda x: x[torch.LongTensor([2,1,0])]), #turn to BGR
                           transforms.Normalize(mean=[0.485, 0.456, 0.406], #subtract imagenet mean
                                                std=[1,1,1]),
                           transforms.Lambda(lambda x: x.mul_(255)),
                          ])
postpa = transforms.Compose([transforms.Lambda(lambda x: x.mul_(1./255)),
                           transforms.Normalize(mean=[-0.485, -0.456, -0.406], #add imagenet mean
                                                std=[1,1,1]),
                     #      transforms.Lambda(lambda x: x[torch.LongTensor([2,1,0])]), #turn to RGB
                           ])
postpb = transforms.Compose([transforms.ToPILImage()])
def postp(tensor): # to clip results in the range [0,1]
    t = postpa(tensor)
    t[t>1] = 1
    t[t<0] = 0
    img = postpb(t)
    return img

In [ ]:
#loss function and transforms:
def Gram_mat(data):
  a,channel,height,width=data.shape
  new_data=data.view(a*channel,height*width)
  gram_val=torch.mm(new_data,new_data.t())/(height*width)

  return gram_val

def content_loss(current,actual):
  #print(current.shape,actual.shape)
  a,channel,height,width=current.shape
  get=torch.norm(current-actual)/(a*channel*height*width) #normalised the values
  #print(current[:,0,:5,:5])
  #print(actual[:,0,:5,:5])

  return get #by default forbenius norm or l2-norm

def style_loss(style,gen):
  a,channels,height,width=style.shape
  style_gram=Gram_mat(style)
  gen_gram=Gram_mat(gen)
  out=torch.norm(style_gram-gen_gram)/(channels**2)

  return out

In [ ]:
class StyleNet(nn.Module):
   def __init__(self):
    super().__init__()
    #PRETRAINED MODEL
    self.pretrained = models.vgg19(weights="DEFAULT")
    for param in self.pretrained.parameters():
      param.requires_grad = False
    #all available hooks
    self.fhooks = []
    # initialising hooks
    self.fhooks.append(self.pretrained._modules['features'][0].register_forward_hook(self.hookfn))
    self.fhooks.append(self.pretrained._modules['features'][2].register_forward_hook(self.hookfn))
    self.fhooks.append(self.pretrained._modules['features'][5].register_forward_hook(self.hookfn))
    self.fhooks.append(self.pretrained._modules['features'][7].register_forward_hook(self.hookfn)) #content loss reference
    self.fhooks.append(self.pretrained._modules['features'][10].register_forward_hook(self.hookfn))

    #store hooks output
    self.selected_out = OrderedDict()
    self.count=0
    self.limit=5 # layers limit ,i.e for how many layer i want the output

   def forward(self,x):
    y=self.pretrained(x)

    return y,self.selected_out

   def hookfn(self,layername,inp,out):
    self.selected_out[self.count]=out
    #print(out[:,0,:5,:5])
    #print(self.count,str(layername)) ,one can try to see in which order these are getting printed
    self.count+=1
    if self.limit==self.count:
      self.count=0

In [ ]:
def optimize(model,device,lr=1e-1,iterations=2000,step_size=40):
  img_dirs = ['/kaggle/input/styles/style_images/', '/kaggle/input/cheecks/']
  img_names = ['vg_houses.jpg', 'Tuebingen_Neckarfront.jpg']
  imgs = [Image.open(img_dirs[i] + name) for i,name in enumerate(img_names)]
  imgs_torch = [prep(img) for img in imgs]  # here rather than using opencv ,we will use Image and apply transforms to it .
  if torch.cuda.is_available():
    imgs_torch = [img.unsqueeze(0).cuda() for img in imgs_torch]
  else:
    imgs_torch = [img.unsqueeze(0) for img in imgs_torch]  # adding an extra dimension and imporoting them to cuda

  style_img, content_img = imgs_torch
  x=content_img.clone() #already on cuda
  x.requires_grad=True  #setitup for gradient optimization .

  #x=torch.randn((1,3,256,256)) # since we want to optimize input not model
  #print(x.is_leaf)
  model=model.to(device)
  #print(x.is_leaf)

  lr=lr


  model.eval()
  #model.requires_grad(False)

  _,style_embed_out=model(style_img) # all for style
  style_embed=[style_embed_out[i] for i in range(5)]

  _,content_embed_out=model(content_img) #4th is for content
  content_embed=content_embed_out[3]

  embed=style_embed_out[0]

  total_loss=0.0
  iterations=iterations
  alpha=1
  beta=1e3

  optimizer=torch.optim.Adam([x],lr=lr)
  #scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = step_size, gamma = gamma)

  #training_loop needs to be written here.
  #lets go with adam optimizer

  for i in range(iterations):
    optimizer.zero_grad()

    _,gen_embed_out=model(x)
    #print(gen_embed_out[3]==content_embed_out[3])
    c_loss=content_loss(gen_embed_out[3],content_embed)
    s_loss=[0.2*(style_loss(gen_embed_out[i],style_embed[i]))  for i in range(5)]

    styleloss=0.0
    for loss in s_loss:
      styleloss+=loss

    total_loss=c_loss*alpha+styleloss*beta
    #print(c_loss,styleloss,s_loss)
    total_loss.backward()
    optimizer.step()


    if i%100 == 99:
      print(total_loss)
  out_img =postp(x.data[0].cpu().squeeze())
  plt.figure(figsize=(7,7))
  plt.imshow(out_img)
    #recreate_image(content_img)
    #recreate_image(style_img)

In [ ]:
model=StyleNet()
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")



In [ ]:
device

In [ ]:
t=optimize(model,device)